#### Import Required Libraries 

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from collections import Counter

#### Load Dataset

In [2]:
# Load dataset
df = pd.read_excel('student_data.xlsx')

# Show all columns when displaying DataFrames
pd.set_option('display.max_columns', None)
df.head()

,student_id,course,student_cohort,academic_status,failed_subjects,study_skills(attended),referral,pp_meeting,self_assessment,readiness_assessment_results,follow_up,follow_up_type,subject_1,subject_1_assess_1,subject_1_assess_2,subject_1_assess_3,subject_1_assess_4,attendance_1,learn_jcu_issues_1,lecturer_referral_1,subject_2,subject_2_assess_1,subject_2_assess_2,subject_2_assess_3,subject_2_assess_4,attendance_2,learn_jcu_issues_2,lecturer_referral_2,subject_3,subject_3_assess_1,subject_3_assess_2,subject_3_assess_3,subject_3_assess_4,attendance_3,learn_jcu_issues_3,lecturer_referral_3,comments,identified_issues,course_group,risk,country
0,1,MBA,Continuing,Conditional,1,Essential Skills,Student Counsellor,Booked,Yes,L/G:9/10 N:5/10 R:8/10,Yes,No Reply,LB5113,89.2,61.1,61.5,70.9,94.7,Access,Concern for Welfare,LB5202,59.5,51.6,58.9,67.1,64.8,Access,Attendance,LB5205,58.5,39.1,65.2,40.8,69.3,Access,Non Submission,Week 6. Student reported working long hours. R...,Poor time management,Non-IT,NaN,Australia
1,2,MBA,Transferred,Satisfactory,0,Referencing,Student Advocate,Not relevant,Yes,L/G:9/10 N:5/10 R:8/10,No,Phone,LB5113,64.0,68.6,71.5,62.8,74.0,Access,Non Submission,LB5202,73.7,73.1,59.8,63.2,71.5,No Access,Concern for Welfare,LB5205,80.0,69.2,55.5,69.0,72.5,No Access,Non Submission,NaN,NaN,Non-IT,NaN,Australia
2,3,MBA,New,Satisfactory,0,Writing,Enrollment,Attended,Yes,L/G:9/10 N:5/10 R:8/10,Yes,Phone,LB5113,37.8,77.2,69.9,60.4,52.4,No Access,Attendance,LB5202,69.3,60.5,72.2,64.7,69.8,No Access,Concern for Welfare,LB5205,25.5,58.8,77.7,73.1,69.5,Access,Attendance,NaN,NaN,Non-IT,NaN,Bangladesh
3,4,MBA,SRI to JCUB,Satisfactory,0,Essential Skills,Student Advocate,Attended,No,L/G:9/10 N:5/10 R:8/10,No,F2F,LB5113,77.4,77.1,59.0,77.0,68.2,Access,Concern for Welfare,LB5202,67.1,82.8,77.3,91.3,81.5,Access,Non Submission,LB5205,70.4,83.3,72.8,71.8,65.7,Access,Concern for Welfare,Week 2. Student did not attend orientation. Co...,Late Enrollment,Non-IT,NaN,Bhutan
4,5,MBA,Return to Study,Academic Caution,1,Essential Skills and Reading,Student Counsellor,Not relevant,No,L/G:9/10 N:5/10 R:8/10,Yes,No Reply,LB5113,62.2,66.1,64.2,52.3,91.4,Access,Attendance,LB5202,65.7,67.2,68.2,74.1,67.8,No Access,Concern for Welfare,LB5205,54.8,55.5,59.6,56.1,35.6,Access,Concern for Welfare,Week 2. Student did not attend orientation. Co...,Late Enrollment,Non-IT,NaN,Bhutan


#### Data Cleaning and Exploration

In [3]:
# Print a concise summary of the DataFrame, including column names, data types, non-null counts, and memory usage.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   student_id                    698 non-null    int64  
 1   course                        698 non-null    object 
 2   student_cohort                698 non-null    object 
 3   academic_status               698 non-null    object 
 4   failed_subjects               698 non-null    int64  
 5   study_skills(attended)        698 non-null    object 
 6   referral                      698 non-null    object 
 7   pp_meeting                    698 non-null    object 
 8   self_assessment               698 non-null    object 
 9   readiness_assessment_results  698 non-null    object 
 10  follow_up                     698 non-null    object 
 11  follow_up_type                698 non-null    object 
 12  subject_1                     698 non-null    object 
 13  subje

In [4]:
# Rename Columns
df.rename(columns={'subject_4_assess_4':'subject_3_assess_4'}, inplace=True)
df.columns

Index(['student_id', 'course', 'student_cohort', 'academic_status',
       'failed_subjects', 'study_skills(attended)', 'referral', 'pp_meeting',
       'self_assessment', 'readiness_assessment_results', 'follow_up',
       'follow_up_type', 'subject_1', 'subject_1_assess_1',
       'subject_1_assess_2', 'subject_1_assess_3', 'subject_1_assess_4',
       'attendance_1', 'learn_jcu_issues_1', 'lecturer_referral_1',
       'subject_2', 'subject_2_assess_1', 'subject_2_assess_2',
       'subject_2_assess_3', 'subject_2_assess_4', 'attendance_2',
       'learn_jcu_issues_2', 'lecturer_referral_2', 'subject_3',
       'subject_3_assess_1', 'subject_3_assess_2', 'subject_3_assess_3',
       'subject_3_assess_4', 'attendance_3', 'learn_jcu_issues_3',
       'lecturer_referral_3', 'comments', 'identified_issues', 'course_group',
       'risk', 'country'],
      dtype='object')

##### Clean Course Data

This section standardizes course names and maps each student's course to a corresponding set of subject codes. It corrects inconsistencies in course naming, then assigns three subject codes per student based on their enrolled course, ensuring a consistent and accurate subject representation in the dataset.

In [5]:
df['course'] = df['course'].str.strip().str.lower()
df['course'].unique()

array(['mba', 'med-mba', 'minftech', 'bbus', 'mengmgmt', 'binftech',
       'mpa', 'mintltourismandhospmgt', 'bthe', 'mpa-mba', 'bcom',
       'mithm-mba', 'mdatasc(prof)', 'minftech-mba', 'pqp(bus)'],
      dtype=object)

In [6]:
# Map subject code to subject names using Dictionary
subject_dict = {
    "BU1002": "Accounting for Decision Making",
    "BU1003": "Principles of Economics",
    "BU1007": "Principles of Data Analysis for Business",
    "BU1112": "Business Law",
    "BX2011": "Foundation of Accounting Principles",
    "BX2014": "Principles of Finance",
    "CP1401": "Fundamentals of Problem Solving and Programming I",
    "CP1402": "Internet Fundamentals",
    "CP1404": "Programming II",
    "TO1008": "Introduction to Tourism, Hospitality and Events Management",
    "TO2117": "Food and Beverage Management",
    "TO3052": "Experience Design for Tourism Hospitality and Events",
    "LB5113": "Corporate Strategy",
    "LB5202": "Marketing Essentials",
    "LB5205": "People in Organisations",
    "MA5831": "Advanced Data Management and Analysis using SAS",
    "MA5840": "Data Science and Strategic Decision Making for Business",
    "MA5851": "Data Science Master Class 1",
    "ED5097": "Research Design and Proposal",
    "ED5880": "Educational Leadership",
    "EG5200": "Career Planning",
    "EG5220": "Advanced Asset Management and Reliability",
    "EG5310": "Professional Placement",
    "CP5046": "ICT Project 1: Analysis and Design",
    "CP5047": "ICT Project 2: Implementation and Commissioning",
    "CP5503": "Enterprise Database Systems - Oracle",
    "TO5101": "Tourism Systems Analysis",
    "TO5103": "Global Destinations and Competitiveness",
    "TO5104": "Tourist Management Strategies",
    "CO5117": "Introduction to Accounting",
    "CO5103": "Management Accounting",
    "CO5109": "Corporate Finance",
    "LB5203": "Sustainable Enterprise",
    "LB5212": "Accounting and Finance for Managers"
}


In [7]:
# Get the subject name from the subject code
subject_dict['BU1007'] # Output: Principles of Data Analysis for Business 

'Principles of Data Analysis for Business'

In [8]:
# Map each course to a list of three subject codes
course_subjects = {
    'Bachelor of Business': ['BU1002', 'BU1003', 'BU1007'],
    'Bachelor of Commerce': ['BU1112', 'BX2011', 'BX2014'],
    'Bachelor of Information Technology': ['CP1401', 'CP1402', 'CP1404'],
    'Bachelor of Tourism, Hospitality and Events': ['TO1008', 'TO2117', 'TO3052'],
    'Master of Business Administration': ['LB5113', 'LB5202', 'LB5205'],
    'Master of Data Science (Professional)': ['MA5831', 'MA5840', 'MA5851'],
    'Master of Education - Master of Business Administration': ['ED5097', 'ED5880', 'LB5113'],
    'Master of Engineering Management': ['EG5200', 'EG5220', 'EG5310'],
    'Master of Information Technology': ['CP5046', 'CP5047', 'CP5503'],
    'Master of Information Technology - Master of Business Administration': ['CP5046', 'LB5113', 'LB5202'],
    'Master of International Tourism and Hospitality Management': ['TO5101', 'TO5103', 'TO5104'],
    'Master of International Tourism and Hospitality Management - Master of Business Administration': ['TO5101', 'LB5113', 'LB5202'],
    'Master of Professional Accounting': ['CO5117', 'CO5103', 'CO5109'],
    'Master of Professional Accounting - Master of Business Administration': ['CO5117', 'CO5103', 'LB5113'],
    'Postgraduate Qualifying Program - Business': ['LB5202', 'LB5203', 'LB5212']
}


In [9]:
# Create a new dictionary with cleaned course names as keys
# For each key-value pair in the original course_subjects dictionary:
#   Remove any leading/trailing spaces from the course name (key) using strip()
#   Convert the course name to lowercase using lower()
#   Keep the value (list of subjects) unchanged
cleaned_course_subjects = {k.strip().lower(): v for k, v in course_subjects.items()}
cleaned_course_subjects

{'bachelor of business': ['BU1002', 'BU1003', 'BU1007'],
 'bachelor of commerce': ['BU1112', 'BX2011', 'BX2014'],
 'bachelor of information technology': ['CP1401', 'CP1402', 'CP1404'],
 'bachelor of tourism, hospitality and events': ['TO1008', 'TO2117', 'TO3052'],
 'master of business administration': ['LB5113', 'LB5202', 'LB5205'],
 'master of data science (professional)': ['MA5831', 'MA5840', 'MA5851'],
 'master of education - master of business administration': ['ED5097',
  'ED5880',
  'LB5113'],
 'master of engineering management': ['EG5200', 'EG5220', 'EG5310'],
 'master of information technology': ['CP5046', 'CP5047', 'CP5503'],
 'master of information technology - master of business administration': ['CP5046',
  'LB5113',
  'LB5202'],
 'master of international tourism and hospitality management': ['TO5101',
  'TO5103',
  'TO5104'],
 'master of international tourism and hospitality management - master of business administration': ['TO5101',
  'LB5113',
  'LB5202'],
 'master of pr

In [10]:
# Get unique course names in DataFrame
unique_courses = df['course'].unique()

# Find which course names are not keys in cleaned_course_subjects dictionary
courses_not_in_cleaned = [course for course in unique_courses if course not in cleaned_course_subjects]
courses_not_in_cleaned

['mba',
 'med-mba',
 'minftech',
 'bbus',
 'mengmgmt',
 'binftech',
 'mpa',
 'mintltourismandhospmgt',
 'bthe',
 'mpa-mba',
 'bcom',
 'mithm-mba',
 'mdatasc(prof)',
 'minftech-mba',
 'pqp(bus)']

In [11]:
# Replace invalid course name to a course name in cleaned course subjects dict
df['course'].replace('master of professional account and master of business administration', 'master of professional accounting - master of business administration', inplace=True)

In [12]:
# check if the course value has been updated
df['course'].unique()

array(['mba', 'med-mba', 'minftech', 'bbus', 'mengmgmt', 'binftech',
       'mpa', 'mintltourismandhospmgt', 'bthe', 'mpa-mba', 'bcom',
       'mithm-mba', 'mdatasc(prof)', 'minftech-mba', 'pqp(bus)'],
      dtype=object)

In [13]:
# Assigns subject codes to each student based on their course.
# For each row, looks up the course in the cleaned_course_subjects dictionary.
# If the course is found, fills subject_1, subject_2, and subject_3 with the corresponding subject codes.
# If the course is not found, fills these columns with None.
def assign_subjects(row):
    subjects = cleaned_course_subjects.get(row['course'], [None, None, None])
    row['subject_1'], row['subject_2'], row['subject_3'] = subjects
    return row

df = df.apply(assign_subjects, axis=1)

In [14]:
# Check for null values in subject 1 to 3
df[['subject_1', 'subject_2', 'subject_3']].isnull().sum()

subject_1    698
subject_2    698
subject_3    698
dtype: int64

In [15]:
df['course'].value_counts()

mba                       102
med-mba                    90
minftech                   77
bbus                       71
mengmgmt                   69
binftech                   56
mpa                        55
mintltourismandhospmgt     50
bthe                       30
mpa-mba                    24
bcom                       22
mithm-mba                  21
mdatasc(prof)              13
minftech-mba               11
pqp(bus)                    7
Name: course, dtype: int64

##### Create a Course Group Data
A new column, Course Group, is created to classify courses into IT and NON-IT categories based on the course names and their content. This categorization helps in segmenting the data for more targeted analysis.

In [16]:
# List of courses that falls under course group 'IT'
it_courses = [ 'master of information technology',
    'bachelor of information technology',
    'master of data science (professional)',
    'master of information technology - master of business administration',
] 

In [17]:
# Create a new column 'course_group' based on whether the course is in the IT list
df['course_group'] = df['course'].apply(
    lambda x: 'IT' if x in it_courses else 'Non-IT'
)


In [18]:
df['course_group'].value_counts()

Non-IT    698
Name: course_group, dtype: int64

##### Clean Academic Status Data

This section establishes student profiles by assigning academic statuses and cohorts. It first designates a fixed number of ‘Excluded’ students within IT and Non-IT groups, then randomly assigns cohorts to others, ensuring specific cohorts only have ‘Satisfactory’ status. Subsequently, remaining academic statuses (‘Satisfactory’, ‘Academic Caution’, and ‘Conditional’) are proportionally allocated between IT and Non-IT students to meet target counts. The process uses controlled randomization to create a realistic and balanced student distribution, finalized with a summary of status counts by course group.

In [19]:
# Reset academic_status
df['academic_status'] = None

# Step 1: Decide total excluded count and IT excluded count
total_excluded = 5
excluded_in_it = 3
excluded_in_non_it = total_excluded - excluded_in_it  # 2

# Step 2: Assign cohorts to all students first (clear student_cohort if needed)
df['student_cohort'] = np.nan  # optional clear

# Find eligible indices by course_group to assign exclusions
it_indices = df[df['course_group'] == 'IT'].index.to_numpy()
non_it_indices = df[df['course_group'] == 'Non-IT'].index.to_numpy()

# Shuffle indices for randomness
np.random.seed(42)
it_excluded_indices = np.random.choice(it_indices, size=excluded_in_it, replace=False)
non_it_excluded_indices = np.random.choice(non_it_indices, size=excluded_in_non_it, replace=False)

# Assign 'Excluded' cohort and status for those selected
df.loc[it_excluded_indices, 'student_cohort'] = 'Excluded'
df.loc[non_it_excluded_indices, 'student_cohort'] = 'Excluded'

# Combine indices using pandas Index union and assign 'Excluded' status
excluded_union = pd.Index(it_excluded_indices).union(pd.Index(non_it_excluded_indices))
df.loc[excluded_union, 'academic_status'] = 'Excluded'

# Step 3: Assign cohorts to the other students (non-excluded)
remaining_indices = df.index.difference(excluded_union)
valid_cohorts = ['Return to Study', 'First year', 'Transferred', 'Continuing', 'LOA', 'New', 'SRI to JCUB']
df.loc[remaining_indices, 'student_cohort'] = np.random.choice(valid_cohorts, size=len(remaining_indices), replace=True)

# Step 4: Assign 'Satisfactory' to cohorts which are only allowed 'Satisfactory'
satisfactory_only_cohorts = ['SRI to JCUB', 'Transferred', 'New']
df.loc[df['student_cohort'].isin(satisfactory_only_cohorts), 'academic_status'] = 'Satisfactory'

# Step 5: Define your target counts
target_satisfactory = 621
target_satisfactory_it = 126
target_satisfactory_non_it = 495
target_ac = 36
target_cond = 36

# Step 6: Calculate remaining 'Satisfactory' to assign (excluding already assigned)
assigned_satisfactory = df['academic_status'].eq('Satisfactory').sum()
remaining_satisfactory = max(target_satisfactory - assigned_satisfactory, 0)

# Step 7: Filter remaining unassigned students excluding excluded cohort and satisfactory-only cohorts
remaining_mask = (
    df['academic_status'].isnull() &
    (~df['student_cohort'].isin(satisfactory_only_cohorts + ['Excluded']))
)
remaining_indices = df[remaining_mask].index

# Split remaining_indices by course_group
remaining_it_indices = df.loc[remaining_indices][df.loc[remaining_indices, 'course_group'] == 'IT'].index
remaining_non_it_indices = df.loc[remaining_indices][df.loc[remaining_indices, 'course_group'] == 'Non-IT'].index

# Calculate how many satisfactory assigned already in each course_group
assigned_satisfactory_it = df[
    (df['academic_status'] == 'Satisfactory') & (df['course_group'] == 'IT')
].shape[0]
assigned_satisfactory_non_it = df[
    (df['academic_status'] == 'Satisfactory') & (df['course_group'] == 'Non-IT')
].shape[0]

# Remaining satisfactory in each course_group
remaining_satisfactory_it = max(target_satisfactory_it - assigned_satisfactory_it, 0)
remaining_satisfactory_non_it = max(target_satisfactory_non_it - assigned_satisfactory_non_it, 0)

# Proportionally split Academic Caution and Conditional statuses
total_remaining_ac_cond = target_ac + target_cond
total_remaining_students = len(remaining_it_indices) + len(remaining_non_it_indices)

prop_it = len(remaining_it_indices) / total_remaining_students if total_remaining_students > 0 else 0
prop_non_it = len(remaining_non_it_indices) / total_remaining_students if total_remaining_students > 0 else 0

ac_it = int(round(target_ac * prop_it))
ac_non_it = target_ac - ac_it

cond_it = int(round(target_cond * prop_it))
cond_non_it = target_cond - cond_it

# Build status lists
status_it = (
    ['Satisfactory'] * remaining_satisfactory_it +
    ['Academic Caution'] * ac_it +
    ['Conditional'] * cond_it
)

status_non_it = (
    ['Satisfactory'] * remaining_satisfactory_non_it +
    ['Academic Caution'] * ac_non_it +
    ['Conditional'] * cond_non_it
)

# Function to adjust length (padding/truncation)
def adjust_length(lst, desired_len, filler):
    if len(lst) > desired_len:
        return lst[:desired_len]
    elif len(lst) < desired_len:
        return lst + [filler] * (desired_len - len(lst))
    else:
        return lst

status_it = adjust_length(status_it, len(remaining_it_indices), 'Academic Caution')
status_non_it = adjust_length(status_non_it, len(remaining_non_it_indices), 'Academic Caution')

# Shuffle status lists
np.random.seed(42)
np.random.shuffle(status_it)
np.random.shuffle(status_non_it)

# Assign statuses back
df.loc[remaining_it_indices, 'academic_status'] = status_it
df.loc[remaining_non_it_indices, 'academic_status'] = status_non_it

# Final checks
#Group by course_group and academic_status and count
status_distribution = df.groupby(['course_group', 'academic_status']).size().unstack(fill_value=0)

print("Academic status distribution by course_group:\n")
print(status_distribution)

ValueError: 'a' cannot be empty unless no samples are taken

##### Clean Failed Subjects Data
This section assigns values to the failed_subjects column by grouping students based on academic status and course group (IT or Non-IT). It sets high failure counts for excluded students and supplements with random selections to meet specific quotas. Most satisfactory students are assigned zero failures, with a few outliers receiving 1 or 2 failed subjects. Remaining students receive mostly 1 or 2 failed subjects, with some zero-value outliers for variation. The process ensures controlled, reproducible, and realistic failure data distributions across groups, finalized with a summary of assigned values.

In [ ]:
# Masks for clarity
mask_satisfactory_it = (df['academic_status'] == 'Satisfactory') & (df['course_group'] == 'IT')
mask_it = (df['course_group'] == 'IT')

mask_satisfactory_non_it = (df['academic_status'] == 'Satisfactory') & (df['course_group'] == 'Non-IT')
mask_non_it = (df['course_group'] == 'Non-IT')

mask_excluded = (df['academic_status'] == 'Excluded')

# Excluded indices
excluded_it_indices = df[mask_excluded & mask_it].index
excluded_non_it_indices = df[mask_excluded & mask_non_it].index

# ---- Assign failed_subjects > 3 ----

# IT group
num_it_excluded = len(excluded_it_indices)
num_it_failed_gt3 = 10
num_it_remaining_gt3 = num_it_failed_gt3 - num_it_excluded

if num_it_remaining_gt3 < 0:
    raise ValueError(f"IT Excluded students ({num_it_excluded}) exceed 10 total failed_subjects>3 limit.")

eligible_it_indices = df[mask_it & ~mask_excluded].index
np.random.seed(42)
it_gt3_extra = np.random.choice(eligible_it_indices, size=num_it_remaining_gt3, replace=False)
df.loc[excluded_it_indices.union(it_gt3_extra), 'failed_subjects'] = 4

# Non-IT group
num_non_it_excluded = len(excluded_non_it_indices)
num_non_it_failed_gt3 = 7
num_non_it_remaining_gt3 = num_non_it_failed_gt3 - num_non_it_excluded

if num_non_it_remaining_gt3 < 0:
    raise ValueError(f"Non-IT Excluded students ({num_non_it_excluded}) exceed 7 total failed_subjects>3 limit.")

eligible_non_it_indices = df[mask_non_it & ~mask_excluded].index
non_it_gt3_extra = np.random.choice(eligible_non_it_indices, size=num_non_it_remaining_gt3, replace=False)
df.loc[excluded_non_it_indices.union(non_it_gt3_extra), 'failed_subjects'] = 4

# ---- IT Satisfactory: assign failed_subjects = 0 to most, leave 2-3 outliers with 1 or 2 ----

satisfactory_it_indices = df[mask_satisfactory_it].index
num_outliers_it_satisfactory = 3

np.random.seed(101)
outliers_it_satisfactory = np.random.choice(satisfactory_it_indices, size=num_outliers_it_satisfactory, replace=False)
df.loc[satisfactory_it_indices.difference(outliers_it_satisfactory), 'failed_subjects'] = 0
df.loc[outliers_it_satisfactory, 'failed_subjects'] = np.random.choice([1, 2], size=num_outliers_it_satisfactory)

# ---- IT remaining students (excluding assigned above) assign failed_subjects 1 or 2 with 2-3 zeros as outliers ----

assigned_it_failed = df.loc[mask_it, 'failed_subjects'].notna()
remaining_it = df[mask_it & ~assigned_it_failed].index
num_zeros_it_remaining = 3

np.random.seed(102)
zeros_it_remaining = np.random.choice(remaining_it, size=num_zeros_it_remaining, replace=False)
df.loc[zeros_it_remaining, 'failed_subjects'] = 0

remaining_it_other = remaining_it.difference(zeros_it_remaining)
df.loc[remaining_it_other, 'failed_subjects'] = np.random.choice([1, 2], size=len(remaining_it_other), replace=True)

# ---- Non-IT Satisfactory: assign failed_subjects = 0 to most, leave 2-3 outliers with 1 or 2 ----

satisfactory_non_it_indices = df[mask_satisfactory_non_it].index
num_outliers_non_it_satisfactory = 3

np.random.seed(103)
outliers_non_it_satisfactory = np.random.choice(satisfactory_non_it_indices, size=num_outliers_non_it_satisfactory, replace=False)
df.loc[satisfactory_non_it_indices.difference(outliers_non_it_satisfactory), 'failed_subjects'] = 0
df.loc[outliers_non_it_satisfactory, 'failed_subjects'] = np.random.choice([1, 2], size=num_outliers_non_it_satisfactory)

# ---- Non-IT remaining students (excluding assigned above) assign failed_subjects 1 or 2 with 2-3 zeros as outliers ----

assigned_non_it_failed = df.loc[mask_non_it, 'failed_subjects'].notna()
remaining_non_it = df[mask_non_it & ~assigned_non_it_failed].index
num_zeros_non_it_remaining = 3

np.random.seed(104)
zeros_non_it_remaining = np.random.choice(remaining_non_it, size=num_zeros_non_it_remaining, replace=False)
df.loc[zeros_non_it_remaining, 'failed_subjects'] = 0

remaining_non_it_other = remaining_non_it.difference(zeros_non_it_remaining)
df.loc[remaining_non_it_other, 'failed_subjects'] = np.random.choice([1, 2], size=len(remaining_non_it_other), replace=True)

# ---- Summary ----

print(f"Total students with failed_subjects > 3: {(df['failed_subjects'] > 3).sum()}")
print(f" - IT failed_subjects > 3: {df[(df['course_group'] == 'IT') & (df['failed_subjects'] > 3)].shape[0]}")
print(f" - Non-IT failed_subjects > 3: {df[(df['course_group'] == 'Non-IT') & (df['failed_subjects'] > 3)].shape[0]}")

print(f"IT Satisfactory students with failed_subjects = 0: {(df[mask_satisfactory_it & (df['failed_subjects'] == 0)]).shape[0]}")
print(f"IT Satisfactory outliers with failed_subjects 1 or 2: {(df[mask_satisfactory_it & (df['failed_subjects'] > 0)]).shape[0]}")

print(f"Non-IT Satisfactory students with failed_subjects = 0: {(df[mask_satisfactory_non_it & (df['failed_subjects'] == 0)]).shape[0]}")
print(f"Non-IT Satisfactory outliers with failed_subjects 1 or 2: {(df[mask_satisfactory_non_it & (df['failed_subjects'] > 0)]).shape[0]}")

print(f"Failed subjects distribution for IT students:\n{df[mask_it]['failed_subjects'].value_counts()}")
print(f"Failed subjects distribution for Non-IT students:\n{df[mask_non_it]['failed_subjects'].value_counts()}")


##### Clean Attendance Data

This section generates attendance percentages for three subjects based on each student’s academic status, cohort classification, and number of failed subjects. Attendance averages are adjusted to reflect realistic behavioral patterns: students with satisfactory academic standing typically exhibit higher attendance rates, whereas those identified as at risk or excluded show lower rates. Additionally, attendance is modulated according to student cohort, with new or returning students generally having reduced attendance compared to continuing students. Attendance values are sampled from normal distributions with these tailored parameters and clipped to remain within the 0-100% range. A fixed random seed ensures replicability of the results. Summary statistics provide insight into average attendance both overall and segmented by academic status and cohort group.

In [ ]:
import numpy as np
import pandas as pd

# Minimum and Maximum attendance threshold
MIN_ATTD_ASSIGN = 0
MAX_ATTD_ASSIGN = 100

def generate_attendance(row):
    """
    Generate realistic attendance for three subjects based on academic status, cohort, and failed_subjects.
    """
    base_mean = 66.5 # global research value for mean attendance
    base_std = 13

    # Academic status logic
    status = row['academic_status']
    if status == 'Satisfactory':
        mean = base_mean + 3
        std = 7
    elif status in ['Academic Caution', 'Conditional']:
        mean = base_mean - 6
        std = 13
    elif status in ['At Risk', 'Excluded']:
        mean = base_mean - 14
        std = 17
    else:
        mean = base_mean
        std = base_std

    # Cohort adjustment
    cohort = row['student_cohort'] if isinstance(row['student_cohort'], str) else ''
    if 'First year' in cohort or 'New' in cohort:
        mean -= 7
    elif 'Return' in cohort or 'Transferred' in cohort:
        mean -= 3
    elif 'Continuing' in cohort or 'LOA' in cohort:
        mean -= 1

    # Adjust for number of failed_subjects (robust to missing/NaN)
    try:
        fails = int(row.get('failed_subjects', 0))
    except:
        fails = 0
    if fails > 3:
        mean -= 12
    elif fails == 0 and status == 'Satisfactory':
        mean += 4  # these are your most regular attenders!

    # Clamp all means within sensible range
    mean = max(MIN_ATTD_ASSIGN, min(MAX_ATTD_ASSIGN, mean))
    std = min(std, 20)  # don't let stdev go excessive

    # Generate attendance for 3 subjects with some random variation
    att1 = np.clip(np.random.normal(loc=mean, scale=std), MIN_ATTD_ASSIGN, MAX_ATTD_ASSIGN)
    att2 = np.clip(np.random.normal(loc=mean + np.random.uniform(-3,3), scale=std), MIN_ATTD_ASSIGN, MAX_ATTD_ASSIGN)
    att3 = np.clip(np.random.normal(loc=mean + np.random.uniform(-3,3), scale=std), MIN_ATTD_ASSIGN, MAX_ATTD_ASSIGN)

    return pd.Series([round(att1,1), round(att2,1), round(att3,1)])

# ---- APPLY TO THE DATAFRAME ----

np.random.seed(101)  # for reproducible results

df[['attendance_1', 'attendance_2', 'attendance_3']] = df.apply(generate_attendance, axis=1)

# ---- CHECK THE RESULTING MEANS ----

print("Mean attendance_1:", df['attendance_1'].mean())
print("Mean attendance_2:", df['attendance_2'].mean())
print("Mean attendance_3:", df['attendance_3'].mean())

# Combined overall mean:
overall_attendance_mean = pd.concat([df['attendance_1'],df['attendance_2'],df['attendance_3']]).mean()
print("Overall attendance mean (all columns):", overall_attendance_mean)

# To see attendance distribution by academic status or cohort:
print("\nAttendance by Academic Status:\n", df.groupby('academic_status')[['attendance_1','attendance_2','attendance_3']].mean())
print("\nAttendance by Student Cohort:\n", df.groupby('student_cohort')[['attendance_1','attendance_2','attendance_3']].mean())


##### Clean Subject Assessments Score

This section assigns realistic assessment scores to each student, with adjustments based on academic status, cohort membership, and attendance patterns, resulting in higher scores for students with stronger academic performance. Subsequently, a small proportion of students (approximately 5%) are randomly selected as outliers and assigned notably low or high scores to introduce additional variability reflecting real-world data distributions. The code also tracks and summarizes the frequency of students appearing as outliers across multiple assessments.

In [ ]:
subject_cols = [f"subject_{i}_assess_{j}" for i in range(1,4) for j in range(1,4)]
print(df[subject_cols].describe().transpose())

In [ ]:

# Define subject assessment columns
subject_cols = [f"subject_{i}_assess_{j}"
                for i in range(1, 4)  # subjects 1 to 3
                for j in range(1, 5)]  # assessments 1 to 4

def assign_assessments(row):
    # Step 1: Immediate 0s for Excluded
    if row['academic_status'] == 'Excluded':
        return pd.Series([0] * len(subject_cols), index=subject_cols)

    # 2. Set base mean & std by prior academic_status
    status = row['academic_status']
    base = {
        'Satisfactory':     (72, 7),
        'Academic Caution': (61, 12),
        'Conditional':      (58, 13),
        'At Risk':          (52, 16)
    }
    mean, std = base.get(status, (60, 12))

    # Cohort adjustments
    cohort = str(row['student_cohort'])
    if 'First year' in cohort or 'New' in cohort:
        mean -= 4
    elif 'Return' in cohort or 'Transferred' in cohort:
        mean -= 2

    # Attendance adjustment: stronger for very low, weaker for medium/high
    attendance_cols = ['attendance_1','attendance_2','attendance_3']
    if set(attendance_cols) <= set(row.index):
        att = float(np.mean([row[col] for col in attendance_cols]))
        if att >= 80:
            mean += 2
        elif att < 60:
            mean -= 3
        elif att < 50:
            mean -= 6
        # else no adjustment
    # Clamp mean within 40-95
    mean = max(40, min(95, mean))
    std = min(std, 20)

    # Generate marks for each assessment (simulate real small ups and downs per assessment)
    scores = []
    for _ in subject_cols:
        score = np.random.normal(loc=mean + np.random.uniform(-2,2), scale=std)
        score = max(0, min(100, round(score, 1)))
        scores.append(score)
    return pd.Series(scores, index=subject_cols)

# Apply to a df
np.random.seed(120)  # For reproducibility
df[subject_cols] = df.apply(assign_assessments, axis=1)

# Quick check
print(df[subject_cols].mean().mean())   # Should be mid 60s, or to your liking
print(df.groupby('academic_status')[subject_cols].mean().mean(axis=1))  # Means per status


In [ ]:
subject_cols = [f"subject_{i}_assess_{j}" for i in range(1,4) for j in range(1,4)]
print(df[subject_cols].describe().transpose())

In [ ]:
def add_outliers(
    df,
    subject_cols,
    low_pct=0.05,
    high_pct=0.05,
    filter_academic_status=None,  # e.g. ['Satisfactory', 'At Risk'], None means all
    filter_cohorts=None           # e.g. ['First year', 'New'], None means all
):

    np.random.seed(999)  # For reproducibility

    # Filter rows
    if filter_academic_status is not None:
        mask_status = df['academic_status'].isin(filter_academic_status)
    else:
        mask_status = pd.Series(True, index=df.index)

    if filter_cohorts is not None:
        mask_cohort = df['student_cohort'].isin(filter_cohorts)
    else:
        mask_cohort = pd.Series(True, index=df.index)

    eligible_indices = df[mask_status & mask_cohort].index.tolist()
    n_students = len(eligible_indices)

    if n_students == 0:
        print("Warning: No students matching the filter criteria for outliers.")
        return df, {}

    outliers_log = {'low_outliers': {}, 'high_outliers': {}}

    for col in subject_cols:
        n_low = int(n_students * low_pct)
        n_high = int(n_students * high_pct)

        # Select random students for low outliers
        low_indices = np.random.choice(eligible_indices, size=n_low, replace=False)
        # Ensure no overlap
        remaining_for_high = list(set(eligible_indices) - set(low_indices))
        n_high = min(n_high, len(remaining_for_high))
        high_indices = np.random.choice(remaining_for_high, size=n_high, replace=False)

        # Assign low scores (uniform 0 to 40)
        low_scores = np.random.uniform(0, 40, size=n_low).round(1)
        df.loc[low_indices, col] = low_scores

        # Assign high scores (uniform 90 to 100)
        high_scores = np.random.uniform(90, 100, size=n_high).round(1)
        df.loc[high_indices, col] = high_scores

        # Log info
        outliers_log['low_outliers'][col] = list(low_indices)
        outliers_log['high_outliers'][col] = list(high_indices)

        print(f"{col}: assigned {n_low} low outliers, {n_high} high outliers")

    return df, outliers_log

# List of assessment columns
subject_cols = [col for col in df.columns if col.startswith('subject_') and 'assess_' in col]

# Add outliers only for Satisfactory students
df, log = add_outliers(
    df,
    subject_cols,
    low_pct=0.05,
    high_pct=0.05,
    filter_academic_status=['Satisfactory'],
    filter_cohorts=None  # Or specify cohorts like ['First year','New']
)

# Check summary of affected rows
total_low_outliers = sum(len(v) for v in log['low_outliers'].values())
total_high_outliers = sum(len(v) for v in log['high_outliers'].values())

print(f"Total low outliers assigned: {total_low_outliers}")
print(f"Total high outliers assigned: {total_high_outliers}")
subject_cols = [f"subject_{i}_assess_{j}"
                for i in range(1, 4)  # subjects 1 to 3
                for j in range(1, 5)]  # assessments 1 to 4

In [ ]:

# Build dictionaries: column -> set(row indices)
low_outlier_indices = {}
high_outlier_indices = {}

for col in subject_cols:
    low_outlier_indices[col] = set(df[df[col] <= 40].index)
    high_outlier_indices[col] = set(df[df[col] >= 90].index)



In [ ]:

# For low outliers:
all_low_indices = [idx for indices in low_outlier_indices.values() for idx in indices]
low_counts = Counter(all_low_indices)

# For high outliers:
all_high_indices = [idx for indices in high_outlier_indices.values() for idx in indices]
high_counts = Counter(all_high_indices)


In [ ]:
# How many students are outliers in 1, 2, ... n columns?
print("Low outliers per student frequency:")
print(Counter(low_counts.values()))
print("\nHigh outliers per student frequency:")
print(Counter(high_counts.values()))


In [ ]:
subject_cols = [f"subject_{i}_assess_{j}" for i in range(1,4) for j in range(1,4)]
print(df[subject_cols].describe().transpose())
